 # importing libraries

In [251]:
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function

import numpy as np
import tensorflow as tf
#from tensorflow.python.keras.layers import Input, Dense
#from tensorflow.python.feature_column import dense_features_v2
#from tensorflow.python.feature_column import dense_features 
#from tensorflow.python.feature_column import feature_column
#from tensorflow.python.feature_column import feature_column_v2
#from tensorflow.python.feature_column import sequence_feature_column
#from tensorflow.python.feature_column import serialization

import re #clean the text, to simplyfy conversation
import time #to measure training time

### part 1 - data preprocessing

### importing the dataset

In [252]:
# alternate method to open file

#import os
#path = '/home/raxit/chatbot/datasets'
#file = 'movie_lines.txt'
#lines = open(os.path.join(path,file))

lines = open('/home/raxit/chatbot/datasets/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('/home/raxit/chatbot/datasets/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')


In [253]:
#lines

In [254]:
#conversations

## creating a dictionary that maps each line and its id

In [255]:
id2line = {}
for line in lines :
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [256]:
#id2line

## creating the list of all conversations

In [298]:
conversations_ids = []
for conversation in conversations [:-1]:
    _conversation = conversation.split (' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations_ids.append(_conversation.split(','))

In [258]:
#conversations_ids



## getting separately questions and answers

In [259]:
questions = [ ]
answers = [ ]
for conversation in conversations_ids : 
    for i in range (len(conversation)-1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [260]:
## questions


## first cleaning of text

In [261]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"can't","cannot",text)
    text = re.sub(r"\"ll","will",text)
    text = re.sub(r"\"ve","have",text)
    text = re.sub(r"\"d","would",text)
    text = re.sub(r"didn't","did not",text)
    text = re.sub(r"[-()\"#*$%&!~|,.?{}+:;<>@/^]","",text)
    return text

## cleaning the questions

In [262]:
clean_questions = []
for question in questions:
    clean_questions.append (clean_text(question))

## cleaning the answers

In [263]:
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [264]:
#clean_questions

In [265]:
#clean_answers

## creating a dictionary that maps each word to its occurance in questions and in answers

In [266]:
word2count = {}  #making word2count dictionary

#for questions

for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count [word] = 1
        else:
            word2count [word] += 1
            
            
#now for answers-
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
        
        

## creating two dictionaries that maps the questions word and answers word to a unique integer
 

In [299]:
    
    #here we are taking a threshold of word occurance in questions and  in answers
  #  if any word is coming below threshold quantity we will exclude that and whose word
   # qunatity is greater than the threshold we will take that word.
    
threshold_questions = 20

In [300]:
#making dictionary for questions

questionswords2int = {}
word_number = 0 #again we are starting from zero
for word,count in word2count.items():
    if count >= threshold_questions:
        questionswords2int[word] = word_number
        word_number += 1
#making dictionary for answers
threshold_answers = 20
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_answers:
        answerswords2int[word] = word_number
        word_number += 1
        
            


## adding the last tokens to the directories

In [301]:
tokens = ["<PAD>","<EOS>","<OUT>","<SOS>"]

for token in tokens:
    questionswords2int[token] = len(questionswords2int) +1
    
for token in tokens:
    answerswords2int[token] = len(answerswords2int) +1

## creating the inverse dictionary for answersword2int dictionary to map the words

In [302]:
# w - word & w_i - word integer
answersints2word = {w_i:w for w,w_i in answerswords2int.items()}

In [303]:
#answersints2word

## adding the end of string token to the end of evry answers

In [304]:
# doing beacause EOS needed in the end of decoding in seq2seq model


for i in range (len(clean_answers)):
    clean_answers[i] += ' <EOS>'

In [305]:
#clean_answers

## translating all the question and answers in to integer

In [306]:
#and replacing all the words that were filtered out by <OUT>
# to sort all the questions and answers by their length to optimize training performance

In [307]:
#list for changing the words of each question into integer value

questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)

## list for changing the words of each answer into integer value

In [308]:
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    answers_into_int.append(ints)

In [309]:
#answers_into_int

In [310]:
#questions_into_int

## sorting question and answers by the length of question

In [311]:
#it will reduce the padding during the training and speedup our training


sorted_clean_questions = []
sorted_clean_answers = []
for length in range (1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [312]:
#sorted_clean_questions

In [313]:
#sorted_clean_answers


# BUILDING SEQ2SEQ MODEL...

## Creating placeholders for the inputs and targets

In [315]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name = "input")#create placeholderr for inputs
    targets = tf.placeholder(tf.int32,[None,None], name  = "target")#create placeholder for targets
# making parameter(hyperparameter[learning_rate & keep_prob]) used to control dropout rate
# dropouts are the rate of neurons that we are choosed to overwrite during one iteration in training
# we are using 20% dropout to deactivate 20% neuron
# we increase or decrease dropout by using keep_prob hyperparameter
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, targets, lr, keep_prob

## preprocessing the targets,

In [316]:
#here we'll make the batches of the targets(answers[must be in batches])
# because decoder will decode the batches.(neural nets wont accept the single target)

#sentence must start from the SOS(start of the sentence), so we have to make SOS in each answers of each batches to proceed.

#so here we create batches and adding SOS tokens.

In [317]:
def preprocess_targets(targets, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0],[batch_size, -1],[1,1])
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets
    #batch_size is no. of lines in batch
    #strided_slice - extract the subset of tensors
    # for horizontal, axis=1 & for vertical, axis=0 

# creating the encoder rnn layer

In [318]:
def encoder_rnn (rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout]*num_layers)
    # num_layers create encoder cell
    # encoder cell consist of several lstm layers
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                    cell_bw = encoder_cell,
                                                                    sequence_length = sequence_length,
                                                                    inputs = rnn_inputs,
                                                                    dtype = tf.float32)
    # bidirectional_dynamic_rnn take input and build independent forward and backward rnn(input of forward and backward must match)
    # _, -indicates that we only want second element returned by this future bidirectional dinemic rnn function
    return encoder_state

## decoding the training set

In [319]:
#for encoder we only need encoder state and for decoder we only need decoder output



In [320]:
def decode_training_set(encoder_state,
                        decoder_cell,
                        decoder_embedded_input,
                        sequence_length,
                        decoding_scope,
                        output_function,
                        keep_prob,
                        batch_size):
#decoder getting encoder_state as a part of input proceed to decoding
#decoder_cell - cell in the rnn of the decoder
#embeddings are useful to reduce the dimensoility of categorical variables
#embeddings - is a mapping from descrete objects(i.e. words) to vectors of real no.s(uniquiley) 
#here we roviding i/p to embeddings
#decoding_scope - advance data structure that wraps TF variables  

    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    #we are processing 3 dimn matrix
    
    #preprocesses the training data
    
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = 'bahdanau', num_units = decoder_cell.output_size)
    # attention_keys  = key to compare with target states
    #attention_values = values to construct the context vector(return by encoder and used by decoder)
    #attention_score_function - used to compute similarity between keys and target_states
    # attentio construct_function - used to built attention state
    
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              name = "attn_dec_train")#this is the name scope for the decoder function
    
    
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder (decoder_cell,
                                                                                                               training_decoder_function,
                                                                                                               decoder_embedded_input,
                                                                                                               sequence_length,
                                                                                                               scope = decoding_scope)
    #now we take the droput of the decoder_output
    
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)

## decoding the test/validation set

In [321]:
# we will not use dropout for the test set predictions
# here we are using inference function to predict the outcome that is not involved in  
#training although we dont backpropagate this,cause we are decoding test set to predict

In [322]:
def decode_test_set(encoder_state,
                    decoder_cell,
                    decoder_embeddings_matrix,
                    sos_id,
                    eos_id,
                    maximum_length,
                    num_words,
                    decoding_scope,
                    output_function,
                    keep_prob,
                    batch_size):
    
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    
    
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = 'bahdanau', num_units = decoder_cell.output_size)
    
    
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                              encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              decoder_embeddings_matrix,
                                                                              sos_id,
                                                                              eos_id,
                                                                              maximum_length,
                                                                              num_words,
                                                                              name = "attn_dec_inf")#this is the name scope for the decoder function
    
    test_predictions, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                                test_decoder_function,
                                                                                                                scope = decoding_scope)
                                                                                              
                                                                                              
                                                                                              
                                                                                            
                                                                  
    return test_predictions
    

## creating the decoder rnn

In [323]:
def decoder_rnn(decoder_embedded_input,
                decoder_embeddings_matrix,
                encoder_state,
                num_words,
                sequence_length,
                rnn_size,
                num_layers,
                word2int,
                keep_prob,
                batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        #create lstm layer
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        #now apply some dropout regulariztion to reduce overfitting and increase the accuracy
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.truncated_normal_initializer(stddev = 0.1) #stddev = standard deviation
        biases = tf.zeros_initializer()
        output_function = lambda X: tf.contrib.layers.fully_connected(X,
                                                                      num_words,
                                                                      None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,
                                                   output_function,
                                                   keep_prob,
                                                   batch_size)
        
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embeddings_matrix, 
                                           word2int['<SOS>'],
                                           word2int['<EOS>'],
                                           sequence_length - 1,
                                           num_words,
                                           decoding_scope,
                                           output_function,
                                           keep_prob,
                                           batch_size)
        
        return training_predictions, test_predictions

# building the final seq2seq model

In [324]:
def seq2seq_model(inputs,
                  targets,
                  keep_prob,
                  batch_size,
                  sequence_length,
                  answers_num_words,
                  questions_num_words,
                  encoder_embedding_size,
                  decoder_embedding_size,
                  rnn_size,
                  num_layers,
                  questionswords2int):
    
    
    encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                              answers_num_words + 1,
                                                              encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocessed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                         decoder_embeddings_matrix, 
                                                         encoder_state,
                                                         questions_num_words,
                                                         sequence_length,
                                                         rnn_size,
                                                         num_layers,
                                                         questionswords2int,
                                                         keep_prob,
                                                         batch_size)
    
    
    return training_predictions, test_predictions

# training the seq2seq model

## setting the Hyperparameter

In [325]:
epochs = 100
batch_size = 64
rnn_size = 512
num_layers = 3
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.01
learning_rate_decay = 0.9
min_learning_rate = 0.0002
keep_probability = 0.5

## defining the session

In [326]:
tf.reset_default_graph()
session = tf.InteractiveSession()

## loading the model inputs

In [327]:
inputs, targets, lr, keep_prob = model_inputs()

## setting a sequence_length

In [328]:
#here we are gonna set a default value to produce when the output is not fed into the RNN

sequence_length = tf.placeholder_with_default(25, None, name = 'sequence_length')

## getting the shape of the Inputs tensor

In [329]:
input_shape = tf.shape(inputs)

## getting the training and test predictions

In [330]:
training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]),
                                                       targets,
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int)